In [37]:
# # Librairies
import os
import pandas as pd
from argparse import ArgumentParser
import sys
import fnmatch
import glob
import numpy as np


# recupérer le nombre de variant optimal du fichier haplo
def optimal_variant(haplo_file):
    numb_OptVar= int(haplo_file.split("_")[1].replace("Variant.csv",''))
    return numb_OptVar


# recupérer les variants majoritaires de chaque fichier haplo
def maj_variants(haplo_file):

    read_file=pd.read_csv(f"{gene_dir}/{haplo_file}", index_col=[0])
    var_pos = read_file["Position"]
    read_file= read_file.drop(columns=["Position"])
    read_file.columns= range(1, len(read_file.columns)+1)
    variants=[]
    for col in read_file.columns:
        variant= read_file[col].tolist()
        variants.append(variant)
    return variants, var_pos


# print("****")
nb_replicate=5
seuil_replicate=4

freq_dir= "/home/osidibe/work/fluGenAvi_fugace/freq_allelic_OSI"

directory = "/home/osidibe/work/fluGenAvi_fugace/Analyse_postDESMAN_P2/DESMAN_iter"
gene = "erm_F__3_M17808"

gene_dir = os.path.join(directory + "/DESMAN_" + gene)

nb_haplo=0
for haplo_file in os.listdir(gene_dir):
    

    if fnmatch.fnmatch(haplo_file, "haplo_*"):

        variants , var_pos = maj_variants(haplo_file)
    

        """ Step_1 : ne pas analyser les fichiers de variants incomplets """

        if (optimal_variant(haplo_file)*int(nb_replicate)) == len(variants):
            # print(f"{haplo_file} bon nombre de variant")
            # print(len(variants))


            """ Step_2 : Garder les haplotypes présents au moins dans 80% des réplicats  """
        
            variants_not_dup = [list(x) for x in set(tuple(x) for x in variants)]
        
            variant_file=pd.DataFrame(var_pos)

            for variant in variants_not_dup:
                # print(variant)
                nb_dup=0
                for var in variants:
                    # print(var)
                    if variant == var:
                        nb_dup+=1
                
                if nb_dup >= seuil_replicate:
                    # variant = pd.Series(variant, index= var_pos)
                    variant_file =pd.concat([variant_file, pd.Series(variant)], axis=1)
                    variant_file.to_csv(f"{gene_dir}/{haplo_file}")
                    
            # print(variant_file)
            nb_haplo+=1

        
        else :
            os.remove(f"{gene_dir}/{haplo_file}")

""" Step_3 : créer un fichier des variants uniques et majoritaires  """

final_variant=pd.DataFrame()
print(nb_haplo)
nb_var=0

for i in range(1, nb_haplo+1):
    j= i+1
    try:
        haplo_file_1= f"haplo_{i}Variant.csv"
        haplo_file_2= f"haplo_{j}Variant.csv"

        variants_1, var_pos_1 = maj_variants(haplo_file_1)
        print(variants_1)
        variants_2, var_pos_2 = maj_variants(haplo_file_2)
        print(variants_2)
        # for var in variants_2:
        commun_variant=[]
        for var in variants_1:
            if var in variants_2:
                commun_variant.append(var)
                print(commun_variant)
            if commun_variant == variants_1:
                print(commun_variant)

            else:
                break

    except FileNotFoundError: 
        print(f"file haplo_{j} not find")
       

for var in variants_2:
    
    final_variant = pd.concat([final_variant, pd.Series(var)], axis=1)
final_variant.to_csv(f"{gene_dir}/final_variant.csv")



""" Step_4: vérifier si les variants majoritaires retenus sont correctements reconstruits """



print("Process done succesfully")





0
Process done succesfully


In [41]:
""" Step_4: vérifier si les variants majoritaires sont correctement reconstruits  """
df=pd.read_csv(f"{gene_dir}/final_variant.csv")
print(df)
print(variant_file)


   Unnamed: 0  0 0.1 0.2
0           0  G   G   A
1           1  C   G   G
2           2  C   C   C
   Position  0  0  0
0       365  G  G  A
1       380  C  G  G
2       739  C  C  C
